In [4]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from PIL import Image
import os
import glob
from scipy.stats import multivariate_normal

def load_images(image_folder, batch_size=500):
    file_paths = glob.glob(os.path.join(image_folder, '*.jpg'))
    for i in range(0, len(file_paths), batch_size):
        batch_paths = file_paths[i:i + batch_size]
        images = np.array([np.array(Image.open(fp).convert('L').resize((128, 128))).flatten() for fp in batch_paths])
        labels = np.array([int(os.path.basename(fp).split('_')[-1].split('.')[0]) for fp in batch_paths])
        yield images, labels

pca_models = {i: PCA(n_components=0.9) for i in range(10)}

image_folder = './smalldata'

# image_folder = './data'
all_images = []
all_labels = []
for images, labels in load_images(image_folder):
    all_images.append(images)
    all_labels.append(labels)
all_images = np.concatenate(all_images)
all_labels = np.concatenate(all_labels)

print(f"Total images loaded: {len(all_images)}")
print(f"Image data shape: {all_images.shape}")

# Splitting data into training and testing
X_train, X_test, y_train, y_test = train_test_split(all_images, all_labels, test_size=0.2, random_state=42)
print("Data split into training and testing sets.")

# Standardize data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print("Data standardized.")

# Fit PCA for each class and find parameters
for digit in range(10):
    digit_indices = y_train == digit
    pca_models[digit].fit(X_train_scaled[digit_indices])
    print(f"PCA for digit {digit} retains {pca_models[digit].n_components_} components.")

# Estimate parameters (mean, covariance) for the first two principal components
params = {}
for digit, pca in pca_models.items():
    transformed_data = pca.transform(X_train_scaled[y_train == digit])
    params[digit] = {
        'mean': np.mean(transformed_data, axis=0),
        'cov': np.cov(transformed_data, rowvar=False)
    }
    print(f"Parameters for digit {digit}: Mean - {params[digit]['mean']}, Covariance - {params[digit]['cov']}")

# Define classification function
def classify(image):
    x = scaler.transform(image.reshape(1, -1))
    likelihoods = []
    for digit, pca in pca_models.items():
        x_transformed = pca.transform(x)
        mean, cov = params[digit]['mean'], params[digit]['cov']
        likelihood = multivariate_normal.pdf(x_transformed, mean=mean, cov=cov)
        likelihoods.append(likelihood)
    return np.argmax(likelihoods)

# Evaluate model
correct = 0
for img, label in zip(X_test_scaled, y_test):
    pred = classify(img)
    if pred == label:
        correct += 1

accuracy = correct / len(X_test_scaled)
print(f"Classification accuracy: {accuracy:.2%}")
print(f"Correct predictions: {correct}")
print(f"Incorrect predictions: {len(X_test_scaled)-correct}")


Total images loaded: 1000
Image data shape: (1000, 16384)
Data split into training and testing sets.
Data standardized.
PCA for digit 0 retains 18 components.
PCA for digit 1 retains 12 components.
PCA for digit 2 retains 25 components.
PCA for digit 3 retains 16 components.
PCA for digit 4 retains 17 components.
PCA for digit 5 retains 18 components.
PCA for digit 6 retains 17 components.
PCA for digit 7 retains 15 components.
PCA for digit 8 retains 19 components.
PCA for digit 9 retains 17 components.
Parameters for digit 0: Mean - [-1.29189588e-15 -1.06120019e-15  2.59532655e-15  2.49151349e-15
  9.57387128e-16  8.70876243e-16  6.89203384e-16 -8.82411027e-16
 -1.97244818e-15 -1.96091339e-16  7.15156650e-16 -4.75809868e-16
 -3.31625059e-17 -1.37263938e-15  1.52547527e-15 -7.18040346e-16
 -9.22782774e-17  7.38226219e-16], Covariance - [[ 1.03928123e+04  5.87836583e-11  1.82018034e-12  5.05811006e-13
   3.19932637e-13 -1.81151245e-14 -4.27006546e-13 -5.34566368e-13
  -4.79216582e-13  